In [1]:
# model3_FIGURE.py
#
# By Murillo F. Rodrigues, 25 August 2025
# Wall Lab, Oregon Health & Science University

from pathlib import Path
import tskit, msprime, pyslim, warnings, os
from timeit import default_timer as timer
import polars as pl
import numpy as np
import altair as alt

alt.data_transformers.enable("vegafusion")

# NOTE: the base repository path needs to be configured for your setup here!
repository_path = Path('/Users/rodrigmu/Documents/SimHumanity')

# set the current working directory to the SimHumanity repository
os.chdir(repository_path)

trees_path = repository_path / "simhumanity_trees_RO"

MU_TOTAL = 2.0e-8
WIN_LEN = 1_000_000

In [2]:
chrom = '1'

In [3]:
# load the tree sequence
ts = tskit.load(trees_path / f"chromosome_{chrom}.trees")
seq_len = ts.sequence_length

In [4]:
# load the recombination map
rec_path = Path(f'stdpopsim extraction/extracted/chr{chrom}_recombination.txt')
rec_df = pl.read_csv(rec_path, has_header=False, new_columns=["pos", "rate"])
ends = np.append(rec_df["pos"].to_numpy()[1:], seq_len)
rec_df = rec_df.with_columns(end = ends)

In [5]:
# fetching windows with super low recombination rates to mask from figure
low_rec_intervals = rec_df.filter(pl.col("rate") < 1e-12).select(["pos", "end"]).to_numpy()

# masking the tree sequence
ts = ts.delete_intervals(low_rec_intervals)

In [6]:
# load the genomic elements
ge_path = Path(f'stdpopsim extraction/extracted/chr{chrom}_genomic_elements.txt')
ge_df = pl.read_csv(ge_path, has_header=False, new_columns=["type", "start", "end"])

# breakpoints to compute diversity within each genomic element
breakpoints = np.append(ge_df["start"].to_numpy(), seq_len)

In [7]:
site_div = ts.diversity(mode='site', windows=breakpoints)

In [8]:
branch_div = ts.diversity(mode='branch', windows=breakpoints)

In [9]:
site_div

array([5.08287038e-04, 8.50036777e-05, 6.88855503e-04, ...,
       2.52440024e-04, 2.34110630e-05, 1.29923120e-04], shape=(36005,))

In [10]:
branch_div*MU_TOTAL

array([0.00050511, 0.0004108 , 0.00066391, ..., 0.00034934, 0.00043491,
       0.00016745], shape=(36005,))

In [11]:
ge_df = ge_df.with_columns(site_div = site_div, branch_div = branch_div, type = pl.when(pl.col("type") == 0).then(pl.lit("neutral")).otherwise(pl.lit("exon")).alias("type")).with_columns(ratio = pl.col("site_div")/pl.col("branch_div"))

In [12]:
ge_df.head()

type,start,end,site_div,branch_div,ratio
str,i64,i64,f64,f64,f64
"""neutral""",0,450738,0.000508,25255.433328,2.0126e-8
"""exon""",450739,451677,0.000085,20540.203866,4.1384e-9
"""neutral""",451678,685714,0.000689,33195.749461,2.0751e-8
"""exon""",685715,686653,0.000075,19130.570717,3.9033e-9
"""neutral""",686654,925940,0.000643,32476.253884,1.9793e-8


In [13]:
ge_df.group_by("type").agg(pl.col("site_div").mean(), (pl.col("branch_div")*MU_TOTAL).mean(), pl.col("ratio").mean())

type,site_div,branch_div,ratio
str,f64,f64,f64
"""neutral""",0.000558,0.000559,1.9853e-8
"""exon""",0.000304,0.000559,1.1163e-8


In [14]:
0.000304/0.000559

0.5438282647584973

In [15]:
1.1163/1.9853

0.5622827784213973

In [16]:
ge_df = ge_df.with_columns(log_ratio = (pl.col("ratio").log10()))

In [17]:
IN_TO_PX = 72
site_boxplot = alt.Chart(ge_df.with_columns(log_site_div = (pl.col("site_div")).log10())).mark_boxplot().encode(
    x = alt.X("type", title="", sort='descending'),
    y = alt.Y("log_site_div", title="Site diversity (log10-scaled)"),
    color = alt.Color("type", title="Genomic element type", sort="descending", legend=None),
).properties(
    width=11*IN_TO_PX*0.4,
    height=8.5*IN_TO_PX*0.4,
)
line = alt.Chart(pl.DataFrame({'y': [np.log10(2e-8)]})).mark_rule(color="#333333", strokeWidth=1, strokeDash=[10, 5]).encode(y='y')
line2 = alt.Chart(pl.DataFrame({'y': [np.log10(1.2e-8)]})).mark_rule(color="#333333", strokeWidth=1, strokeDash=[10, 5]).encode(y='y')
site_boxplot# + line + line2
site_boxplot.save("site_boxplot.pdf")

In [18]:
IN_TO_PX = 72
branch_boxplot = alt.Chart(ge_df.filter(pl.col("branch_div") > 0.01).with_columns(log_branch_div = (pl.col("branch_div")).log10())).mark_boxplot().encode(
    x = alt.X("type", title="", sort='descending'),
    y = alt.Y("log_branch_div", title="Branch diversity (log10-scaled)", scale=alt.Scale(domain=[2, 6])),
    color = alt.Color("type", title="Genomic element type", sort="descending", legend=None),
).properties(
    width=11*IN_TO_PX*0.4,
    height=8.5*IN_TO_PX*0.4,
)
branch_boxplot.save("branch_boxplot.pdf")

In [19]:
breakpoints = np.arange(0,seq_len, step=WIN_LEN)
breakpoints = np.append(breakpoints, seq_len)

In [20]:
site_div = ts.diversity(windows=breakpoints, mode="site")
branch_div = ts.diversity(windows=breakpoints, mode="branch")

In [21]:
df = pl.DataFrame({"start":breakpoints[:-1], "end":breakpoints[1:], "site_div":site_div, "branch_div":branch_div})
df = df.filter(pl.col("branch_div") > 0.01).with_columns(ratio = pl.col("site_div")/pl.col("branch_div"))


In [22]:
alt.Chart(df).mark_line().encode(
    x = alt.X("start", title="Position"),
    y = alt.Y("site_div", title="Site diversity"),
).properties(
    width=11*IN_TO_PX*0.5,
    height=8.5*IN_TO_PX*0.5,
)

alt.Chart(...)

In [23]:
alt.Chart(df).mark_line().encode(
    x = alt.X("start", title="Position"),
    y = alt.Y("branch_div", title="Branch diversity"),
).properties(
    width=11*IN_TO_PX*0.5,
    height=8.5*IN_TO_PX*0.5,
)

alt.Chart(...)

In [29]:
ratio_plot = alt.Chart(df).mark_line().encode(
    x = alt.X("start", title="Position"),
    y = alt.Y("ratio", title="Site/Branch Diversity Ratio", axis=alt.Axis(format=".1e"), scale=alt.Scale(domain=[1.8e-8, 2.2e-8])),
).properties(
    width=11*IN_TO_PX*0.8,
    height=8.5*IN_TO_PX*0.4,
)
ratio_plot.save("ratio_plot.pdf")

In [37]:
import site


combined = alt.vconcat(site_boxplot.properties(title='A') | branch_boxplot.properties(title='B') , ratio_plot.properties(title='C') , center=True).configure_title(anchor='start')
combined.save("combined.png", ppi=400)
combined

alt.VConcatChart(...)